In [2]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 32.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompat

In [1]:
import gensim.downloader
from gensim.models import Word2Vec

In [10]:
# Download the pre-trained model
model_1= gensim.downloader.load('word2vec-google-news-300')

In [6]:
import re
import nltk
import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def pre_processing(sentences):
  # lower casing
  for sentence in sentences:
    sentence= sentence.lower()

  # removing punctuations and stopwords
  for sentence in sentences:
    punctuations= r'[^\w\s]'
    sentence= re.sub(punctuations, '', sentence)

  # remove stopwords
  for sentence in sentences:
    stop_words_set= set(stopwords.words('english'))
    word_dict= sentence.split()
    filtered_text= [word for word in word_dict if word not in stop_words_set]
    sentence= ' '.join(filtered_text)

  return sentences

In [8]:
def headline_to_vector_W2V(sentences):
    vector_map= []
    for sentence in sentences:
        vec= []
        for word in sentence.split():
            try:
                vec.append(model_1[word])
            except KeyError:
                continue  # Skip words not in the model
        if vec:
            vector_map.append(np.mean(vec, axis=0))
        else:
            # If no words are found, append a zero vector of the correct size
            vector_map.append(np.zeros(model_1.vector_size))

    return vector_map

In [35]:
import json

f=open("/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json","r")

sentences= []
labels= []
urls= []

for line in f:
    item=json.loads(line)
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

print("done")

done


In [41]:
sentences= pre_processing(sentences)
x= headline_to_vector_W2V(sentences)

x_= np.array(x)
print(x_.shape)
print(x_[0].shape)
print(type(x_))

y =np.array(labels)
print(y.shape)

(28619, 300)
(300,)
<class 'numpy.ndarray'>
(28619,)


In [13]:
import re
import nltk
import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
from sklearn.model_selection import train_test_split as tts
x_train, x_test, train_label, test_label= tts(x_, y, test_size=0.2, random_state=42)

print(type(x_train))
print(type(train_label))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [20]:
input_dim= 300
output_dim= 100

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

model = Sequential([
    Dense(128, activation='relu', input_shape=(300,)),  
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),                   
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu'),                      
    Dense(1, activation='sigmoid')                   
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  
    metrics=['accuracy']
)

In [43]:
from sklearn.model_selection import train_test_split as tts
X_train, X_val, Y_train, Y_val= tts(x_train, train_label, test_size=0.2, random_state=42)

print(X_train.shape)

(18316, 300)


In [44]:
history = model.fit(
    X_train,           
    Y_train,           
    epochs=20,        
    batch_size=32,     
    validation_data=(X_val, Y_val)  
)

Epoch 1/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.6273 - loss: 0.7012 - val_accuracy: 0.7443 - val_loss: 0.5266
Epoch 2/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7408 - loss: 0.5304 - val_accuracy: 0.7816 - val_loss: 0.4698
Epoch 3/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7673 - loss: 0.4947 - val_accuracy: 0.7945 - val_loss: 0.4548
Epoch 4/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7763 - loss: 0.4759 - val_accuracy: 0.7917 - val_loss: 0.4532
Epoch 5/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7914 - loss: 0.4639 - val_accuracy: 0.7923 - val_loss: 0.4430
Epoch 6/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7977 - loss: 0.4452 - val_accuracy: 0.8006 - val_loss: 0.4359
Epoch 7/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7976 - loss: 0.4393 - val_accuracy: 0.7971 - val_loss: 0.4379
Epoch 8/20
573/573 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8022 - loss: 0.4361 - val_accuracy: 0.

In [45]:
probs = model.predict(x_test)
predictions = (probs > 0.5).astype(int)
accuracy = (predictions.flatten() == test_label).mean()
print("Test accuracy:", accuracy)

179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Test accuracy: 0.809748427672956


In [82]:
def predict(sentence, model):
  sentences= [sentence]
  sentences= pre_processing(sentences)
  vectors= headline_to_vector_W2V(sentences)
  vectors=np.array(vectors)
    
  model= model
    
  probs = model.predict(vectors)
  predictions = (probs > 0.5).astype(int)
  if predictions[0][0]==1:
    print(f"{sentence} is --> Sarcastic\n")
  else:
    print(f"{sentence} is --> Non-Sarcastic\n") 

print('done')

done


In [83]:
# lets test
sentences=["Dooms day clock of death makes me happy", "I would fall for a bald man XOXO", "I like to study", "I will kill u harry, dumbeldor said calmly",
           "Ofcourse i like to get beaten!!"]

for sentence in sentences:
  predict(sentence, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Dooms day clock of death makes me happy is --> Sarcastic

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
I would fall for a bald man XOXO is --> Sarcastic

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
I like to study is --> Sarcastic

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
I will kill u harry, dumbeldor said calmly is --> Non-Sarcastic

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Ofcourse i like to get beaten!! is --> Non-Sarcastic

